In [1]:
from __future__ import annotations
import numpy as np
import casadi as ca
import scipy.integrate
import matplotlib.pyplot as plt

In [ ]:
def thrust(throttle, rho, r, V, kV, CT, Cm):
    """
    throttle: 0-1 controls voltage, []
    rho: density of air, [kg/m^3]
    r: radius of propeller, [m]
    V: voltage of battery [V]
    kV: motor kV constant, [rpm/V]
    """
    omega = throttle*V*kV*(2*ca.pi/60)
    q = 0.5*rho*(omega*r)**2
    s = ca.pi*r**2

    return CT*q*s,Cm*q*s

def euler_kinematics(e, w):
    '''Derivative of Euler angles'''
    v = ca.SX.sym('v',3)
    v[0] = (w[1]*ca.sin(e[2])+w[2]*ca.cos(e[2]))/ca.cos(e[1])
    v[1] = w[1]*ca.cos(e[2]) - w[2]*ca.sin(e[2])
    v[2] = w[0] + (w[1]*ca.sin(e[2]+w[2]*ca.cos(e[2])))*ca.tan(e[1])
    return v

def euler_to_dcm(e):
    ''' Transition from Euler angles to direction cosine matrix'''
    phi = e[0]
    theta = e[1]
    psi = e[2]
    R_z = ca.DM([
        [ca.cos(psi), -ca.sin(psi), 0],
        [ca.sin(psi), ca.cos(psi), 0],
        [0, 0, 1]
    ])
    R_y = ca.DM([
        [ca.cos(theta), 0, ca.sin(theta)],
        [0, 1, 0],
        [-ca.sin(theta), 0, ca.cos(theta)]
    ])
    R_x = ca.DM([
        [1, 0, 0],
        [0, ca.cos(phi), -ca.sin(phi)],
        [0, ca.sin(phi), ca.cos(phi)]
    ])

    R_zy = ca.mtimes(Rz, Ry)
    R = ca.mtimes(R_zy, R_x)
    
    return R

def motor2mix(u_motor):
        '''from each motor duty to effect toward control angles'''
    map = np.array([
        [-1,1,1,-1],
        [1,-1,1,-1],
        [1,1,-1,-1],
        [1,1,1,1]])
    return ca.mtimes(map,u_motor)

def mix2motor(u_mix):
    '''from input of control to each motor duty'''
    map = np.linalg.inv(np.array([
        [-1,1,1,-1],
        [1,-1,1,-1],
        [1,1,-1,-1],
        [1,1,1,1]]))
    return ca.mtimes(map,u_mix)

def saturate(motor: ca.SX, len: int) -> ca.SX:
    ''' saturate the input motor voltage '''
    sat = ca.SX.sym('sat', 4)
    for i in range(4):
        v = sat[i]
        v = ca.if_else(v > 1, 1, ca.if_else(v < 0, 0, v))
        sat[i] = v
    return sat

In [ ]:
def derive_equations():
    # Parameters & rotorcraft data
    arm_angles_deg = [45, -135, -45, 135]
    arm_angles_rad = [
        (np.pi / 180) * i for i in arm_angles_deg
    ]
    motor_dirs = [1, 1, -1, -1]             # motor rotation direction
    # l_arm = 0.5                             # length or the rotor arm [m]
    # m = 1                                   # mass of vehicle [kg]
    rho = 1.225                             # density of air [kg/m^3]
    # r = 0.05                                # radius of propeller
    V = 11.1                                # voltage of battery [V]
    kV = 1550                               # motor kV constant, [rpm/V]
    CT = 1.0e-2                             # Thrust coeff
    Cm = 1e-4                               # moment coeff
    # CT = 2.7e-4, Cm = 5e-5, r = 0.1778/2 other option

    # state (x)
    x = ca.SX.sym('x',12)
    omega_b = x[0:3]                       # Angular velocity (body)
    vel_b = x[3:6]                         # Velocity (body)
    pos_n = x[6:9]                         # Position (inertial)
    euler = x[9:12]                        # Orientation (inertial) = r_nb

    # input
    n_motors = len(arm_angles_deg)
    u_mix = ca.SX.sym('u_mix', 4)
    u0 = np.array([0, 0, 0, 0.5])       # roll, pitch, yaw, throttle

    # parameters
    m = ca.SX.sym('m')
    l_arm = ca.SX.sym('l_arm')
    r = ca.SX.sym('r')
    Jx = ca.SX.sym('Jx')
    Jy = ca.SX.sym('Jy')
    Jz = ca.SX.sym('Jz')
    p = ca.vertcat(m, l_arm, r, Jx, Jy, Jz)


    J = ca.diag(ca.vertcat(Jx, Jy, Jz))                                             # Moment of inertia of quadrotor

    # forces and moments
    C_nb = euler_to_dcm(euler)                                                      # from euler to direction cosine matrix
    F_b = ca.mtimes(C_nb, ca.vertcat(0, 0, m*g))                                    # Body force Initialize
    M_b = ca.SX.zeros(3)                                                            # Body moment(torque) Initialize
    u_motor = saturate(mix2motor(u_mix), len(motor_dirs))                           # convert u_mix(angle input) to motor duty

    for i in range(n_motors):
        ri_b = ca.vertcat(
            l_arm*ca.cos(arm_angles_rad[i]),
            l_arm*ca.sin(arm_angles_rad[i]), 0
        )                                                                           # vector to each motor from center
        Fi_b, Mi_b = thrust(
            throttle = u_motor[i], rho, r, V, kV, CT, Cm                            # get scalar F and M of each rotor
        )                                     
        Fi_b_vec = ca.vertcat(0, 0, Fi_b)
        Mi_b_vec = ca.vertcat(0, 0, motor_dirs[i] * Mi_b)                           # get each rotor's F and M vector
        F_b += Fi_b_vec                                                             # sum up all rotor F vector
        M_b += Mi_b_vec + ca.cross(ri_b, Fi_b_vec)                                  # sum up all rotor M vector & M from each leg

    rhs = ca.Function('rhs',[x,u_mix,p],[ca.vertcat(
        ca.mtimes(ca.inv(J_b), M_b - ca.cross(omega_b, ca.mtimes(J_b, omega_b))),   # omega dot (angular acceleration)
        F_b/m - ca.cross(omega_b,vel_b),                                            # v dot (acceleration)
        euler_kinematics(euler,omega_b),                                            # omega (angular velocity) (inertial)
        ca.mtimes(C_nb, vel_b),                                                     # v (velocity) (inertial)
        )], ['x','u_mix','p'],['x_dot'])


    # prelim controller
    roll_cmd = 0.2
    pitch_cmd = 0.2
    yaw_cmd = 0.2

    u_control[0] = roll_mix
    u_control[1] = pitch_mix
    u_control[2] = yaw_mix
    u_control[3] = 0.515
    control = ca.Function('control', [x, p], [u_control], ['x', 'p'], ['u'])

    # rhs = ca.Function('rhs', [x, u, p], [x_dot], ['x', 'u', 'p'], ['x_dot'])
    # shadow_if_necessary = ca.Function('shadow_if_necessary', [r_nb], [so3.Mrp.shadow_if_necessary(r_nb)], ['r_nb'], ['r2_nb'])
    
    # mrp_to_euler = ca.Function('mrp_to_euler', [r_nb], [so3.Euler.from_mrp(r_nb)], ['r_nb'], ['euler'])
    
    # x_sat = ca.SX(x)
    # for i in range(4):
    #     v = x_sat[i + 13]
    #     v = ca.if_else(v > 1, 1, ca.if_else(v < 0, 0, v))
    #     x_sat[i + 13] = v
    # # saturate = ca.Function('saturate', [x], [x_sat], ['x'], ['x_sat'])

    return {
        'x': x,
        'u_mix': u_mix,
        'p': p,
        'rhs': rhs
    }

In [ ]:
def simulate(eqs: dict, dt: float, tf: float):
    x0 = [0]*12
    p0 = [1, 0.5, 0.05, 1, 1, 1]
    u0 = [0.5,0.5,0.5,0.5]
    t_vect = np.arange(0, tf, dt)
    xi = x0
    data = {
        't': [],
        'x': [],
        'u': [],
        # 'euler': [],
    }
    for t in t_vect:
        # u = np.array(eqs['control'](xi, p0)).reshape(-1)

        data['x'].append(xi)
        data['u'].append(u)
        data['t'].append(t)
        # data['euler'].append(np.array(eqs['mrp_to_euler'](xi[6:10])).reshape(-1))
        
        res = scipy.integrate.solve_ivp(fun=lambda t, x: np.array(eqs['rhs'](xi, u0, p0)).reshape(-1), t_span=[t, t+dt], t_eval=[t+dt], y0=xi)
        xi = res['y']
        # xi[6:10] = eqs['shadow_if_necessary'](xi[6:10])
        # xi = eqs['saturate'](xi)
        xi = np.array(xi).reshape(-1)

    for k in data.keys():
        data[k] = np.array(data[k])
    return data

In [ ]:
if __name__ == "__main__":
    quad_model = derive_equations()
    data = simulate(quad_model, 0.01, 10)